# Kernel 1: Naive Kernel
So, this is where it begins. The descent into madness.

So now that are familiar with basics of how the execution model works, where and how a CUDA kernel is launched, and how its result is recevied by the CPU, let's take a look at the actual first (and worst) kernel for General Matrix-Matrix Multiplication.

Small easter egg, the kernel we first launched in the previous notebook is the one we will be defining now, so the arguments we were arbitrarily passing before will make more sense now.


The actual kernel invocation + execution configration parameters set-up:

In [ ]:
// create as many blocks as necessary to map all of C
dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32), 1);  

// 32 * 32 = 1024 threads per block
dim3 blockDim(32, 32, 1);

sgemm_naive<<<gridDim, blockDim>>>(M, N, K, alpha, A, B, beta, C);


The way our execution model works here, is that we assign each thread one entry of our result matrix C, essentially giving each mini worker one allocated spot (element) they must compute and fill in. Based on the block dimension we defined, we have 1024 threads per block, which means each block covers 1024 entries of the matrix. This essentially means that we need total dimensions of the matrix / 1024 blocks to get all the computation needed.

Assume we know that the dimensions of our output matrix C are M x N (will be illuminated later). 

If you're thinking (I did initially) it'd be equivalent to just have gridDim( CEIL_DIV(M, 1024), N, 1); since that will also give us the same number of blocks, unfortunately, that won't work. This is as the layout of the blocks over the 2D matrix must match our thread indexing math - which I will talk about in the next code snippet, finally looking into the black box of our kernel.


In [ ]:
_global__ void sgemm_naive(int M, int N, int K, float alpha, const float* A, const float* B, float beta, float* C) {
// we are computing A x B and storing in C
// A is (M x K) and B is (K x N), thus C is (M x N)
// alpha and beta will be discussed later - but remember the discussion on BLAS prior



// compute position in C that this thread is responsible for
const uint col = blockIdx.x * blockDim.x + threadIdx.x;         // threadIdx.x and threadIdx.y will vary from 0 to 31 
const uint row = blockIdx.y * blockDim.y + threadIdx.y;         // blockIdx.x and blockIdx.y will vary from 0 to CEIL_DIV(N,32) and CEIL_DIV(M,32) respectively

...
}

So as you can see, a kernel is not too far off from regular functions we are used to, having the name defined with the list of arguments. The `__global__` keyword you're going to be seeing a lot of is a **function qualifier** that tells the compiler that the function is not any regular old function, it's a **kernel** - launched from the host (CPU) code but ran on the device (GPU). It then allows us to call the kernel from CPU code, using that <<< >>> triple-chevron launch syntax you should be familiar with at this point.

Kernels always have a return type of void, since they can't return values directly. Instead, they write results to GPU memory, which can be copied back to CPU memory using cudaMemcpy(...). I hope these recurring concepts and keywords are steadily working their way into your mental muscle memory, building a natural familiarity with CUDA.

#### Thread Numbering
Now, above I mentioned, that we couldn't do a less sophisticated approach to the grid dimensions because of thread numbering that we see above. This was something that took me a while to get my head around so I will do you the favour of avoiding a stiff neck by explaining it in a way that clicked to me - and then how it ties in to the dimension defintion above.

**Key Concept**: CUDA code is written from the perspective of a single thread. This is not just an attempt to personify our mini workers, rather, this means that the code is executed independently (and uniquely) by each thread, and built-in variables such as blockIdx, and threadIdx will hold different values depending on which thread is running it.

Since we defined our blockDim.x and blockDim.y both as being 32, our threadId.x and threadId.y are the corresponding threads essentially also holding 32 possible values - ranging from 0 to 31 ~ allowing the 1024 threads to have unique a (threadId.x,threadId.y)

Similarly, based on our gridDim definition, blockId.x and blockId.y will vary from 0 to M/32 and N/32, respectively.

Using these thread-specific, runtime-determined variables i.e. threadIdx telling us where a thread sits in a block and blockIdx telling us which block it belongs to in a grid, combined with the known dimentions of each block, we can calculate a global index for the thread ~ essentially assign it a unique entry in our result matrix that it is responsible for computing.

If you are familiar with accessing using pointer arithmetic, this will feel a lot similar to stuff you have worked with before.

We are essentially trying to find the exact (row, column) index on the result matrix for each thread.

### Another great image from Simon

![](../../images/GEMM1/naivekernel.png)

This image, as effective as it is, might be a bit overwhelming to intially get the message across, so I invite the user to look at this slightly simpler image included with am as-simple example, and then circle back to the more detailed, <> one above.

example: Let's say our output matrix is of dimension M = 4 x N = 4, so here:

![](../../images/GEMM1/index1.png)

Let's say that our objective here is also to assign each (of the 16) entries to a single thread in our grid.

Now, let's say we get an itch to make our block size 2x2, inhabiting each block with 4 threads

<sub>Try to not lower your gaze any more after reading this and mentally think of the code to achieve the above line i.e. defining each block as having 9 threads<sub>

In [ ]:
dim3 blockDim(2, 2, 1);

Now, knowing that each of our blocks can handle 4 matrix entries, we just need (total elements / 4) number of blocks, which we can visualize the mapping of the blocks onto our output matrix as such:

![](../../images/GEMM1/index2.png)

However, these blocks will actually be referenced by their row (y-dimension) and column (x-dimension) rather than by a one-dimensional index, and this can be seen by our grid definition of these blocks

In [ ]:
dim3 gridDim(2, 2, 1);
# which could have been done as
dim3 gridDim(4/2, 4/2, 1);
# and since we know our matrix has N = 4, M = 4, it is the same as
dim3 gridDim( CEIL_DIV(M,2), CEIL_DIV(N,2), 1)

So, after this code, the blocks are defined as such:

![](../../images/GEMM1/index3.png)

To now illustrate why simply setting gridDim( CEIL_DIV(M,4), N, 1) won't work (as promised to be explained above), as even this does technically create the same number of blocks -- the 4 needed to map onto this entire matrix, let's take a look at what that would look like.

So for the following (naive) code:

In [ ]:
dim3 gridDim(CEIL_DIV(M,4), N, 1) 
// this would mean, for our example, we are setting a grid of (1,4,1)

The block mapping created would be the following:

![](../../images/GEMM1/index4.png)

As we can clearly see, our blocks will now both over-cover and under-cover the matrix, skipping out parts. While this seems simplistic, since our actual threads will be assigned the entries based on the block dimensions, and their position in the blocks, this leads to wonky behaviour where threads are assigned index entries that do not exist in the matrix. Clearly, this is not acceptable for our goal of seeking optimal matrix multiplication!

To further illuminate, and wrap up this pesky indexing saga that probably brings back memories from a Computer Systems course you did, lets show how the actual entry assigning looks.

Let's start by taking a look again at the code responsible for this:

In [ ]:
const uint col = blockIdx.x * blockDim.x + threadIdx.x;          
const uint row = blockIdx.y * blockDim.y + threadIdx.y;       

So, now let's sketch this out to ensure our intuition is exactly where it needs to be. Let's pick our lucky little thread as the second thread in the second block i.e. this little guy:

![](../../images/GEMM1/index5.png)

I hope this gif I made is a good display of what's happening, to calculate the column

![](../../images/GEMM1/indexcol.gif)

Follow-up gif for finding the row, and in turn the exact entry that will be assigned to that thread

![](../../images/GEMM1/indexrow.gif)

Finally! We have found both the column and the row (dot-product of which will be found) for the entry that thread (1,0) in Block (1,0) is responsible for. To clarify, this exercise was not to find the thread itself (which kinda seems to be the case since it is mapped exactly onto the assigned entry) but rather to find the element assigned to the thread, knowing that the col and row calculation is done from each thread's perspective distinctly.

![](../../images/GEMM1/index6.png)

One final thing to point out is that this now corresponds to entry [0][3] (zero-indexed) of the matrix which is the 4th column of the first row. A confusion point for me was thinking, grid-wise, this is (0, 3), when **actually it is (3, 0)**. This is as the rows correspond to the y=direction since they are vertical, and the columns correspond to the x-direction, being horizontal thus following the (x, y) standard.


Whoo! Okay, mastering indexing: checked!
That should hopefully keep us at bay from being confused by any other indexing sections in the upcoming kernels (spoiler: EVERY one of them has it).



### Naive Kernel Inner Working
So after the thread numbering has been dealt with, with elements of the output matrix assigned, then we have the simplistic code to actually compute the values:

In [ ]:
if (col < M && row < N) {    
    // ... main body of code is here
}

The condition is used to ensure that our row and column never goes out of bounds of the matrix, which might happen if our M or N are not exact multiples of 32 ~ recall CEIL_DIV(M, 32) and CEIL_DIV(N, 32).

Then, inside the condition:

In [ ]:
    float tmp = 0.0;
    for (int i = 0; i < K; i++){
        tmp += A[row * K + i] * B[i * N + col]
    }   


After the tmp value, which will hold our output value, is initialized, it is incremented by the products of the elements in the calculated col and row - effectively performing a dot product of the  **col** and  **row**.

Major thing to note is that matrices are stored as 1D arrays in row-major order, which should rid that creeping itch on your head about the dot product computation. So simply, for matrix A, each row has K elements so we just skip to the row in question and iterate through the elements using i, and for matrix B, each row has N elements, so we iterate through each row, and just grab the element from the wanted column in each row.

Small gif below to quickly illustrate this, to round off this refresher on indexing:

![](../../images/GEMM1/dotproduct.gif)

Then, finally, to actually populate the output matrix with this new value 

In [ ]:
C[row * N + col] = alpha * tmp + beta * C[row * N + col];

Heh. You were probably expecting a simple matrix entry assignment e.g. C[row * N + col] = tmp, and ready to jump on to the next kernel. Well, remember our little detour into BLAS and how it standarizes linear algebra subproblems? We still need to follow the contract given to us.

The weird assignment, shown before in blas_preface.ipynb, probably did not make much sense, and truthfully, it's not much clearer now.
So allow me to shed light on what's happening here

The keyword here, although a boring one is: **efficiency**, with the motivation being to do more in a single pass
For instance, if they standardized SGEMM as just C = A x B, then:
    - If we needed to scale the result by 2 i.e. C = 2AB, there'd be a sepearate pass to multiply the result by 2
    - If we wanted to add the result onto an existing matrix i.e. C = AB + C_old, there'd be a seperate pass to add them together

By having these additional alpha and beta scalars, we can do all this in the same pass, saving computation time.

Further, using these scalars, we can also mould them to special cases
    - alpha = 1, beta = 0 results in C = A x B
    - alpha = 1, beta = 1 results in C += A x B
    - and for any custom weighted blending we can do C = alpha(AxB) + beta(C)

Knowing this, we can see that the assignment used in this kernel can be customised to harness any of these cases, based on whatever argument values will be passed from the CPU-side (host) code that calls the kernel,

### Efficiency Check
At this point, you're probably thinking "Yeah, makes sense. I see how this would result in correct matrix multiplication", and you would not be wrong, but our focus here is on performance, not correctness.

As you know, this is our first (and worst) kernel, and it's probably not clear why this is so bad.

It can be boiled down to slow, redundant memory accesses. If we take a quick look back at our simple example:

![](../../images/GEMM1/index7.png)

Focusing on the top-right block (Block (1,0)), if we were to compute the entries for threads (1,0) and (1,1) we would use the same column (3) of B but different rows of A (0 and 1). With our naive kernel, this reusults in a very inefficient memory access pattern, where the column would be fetched repeatedly from global memory (very slow read) when it could have been stored in the block's shared memory (on-chip and 100x faster) and reused. 

Clearly, if we scale this up to happening for each and every single thread i.e. re-reading previously data from global memory, bandwidth wastage is at a high.

You can probably deduce that here is a lot of improvement work to be done. 

When you're ready, refill that coffee and jump into the next kernel.

prompt theory

// create as many blocks as necessary to map all of C dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32), 1); // 32 * 32 = 1024 threads per block dim3 blockDim(32, 32, 1); do you remember our conversation about how I thought how since this just corresponds to each block having 1024 threads, and each thread handling one entry in the output matrix which means each block handles 1024 entries, we are just defining the grid as having M*N / 1024 which is done as M/32 * N/32 in the code above, but could it also be done as M/1024, N, 1 which would give an equivalent amount of blocks to handle the matrix

// wait so now ur saying stuff like numpy pytorch etc actually uses GPU-performant matrix multiplication without us even knowing or explicitliy giving permission ? it doesnt do any of that on the CPU

// wait so if the implementations can be optimized and customized, explain what exactly is being standardized by BLAS - what is specification of these linear algeba subproblems even mean ?